In [21]:
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas
from icecream import ic
from bokeh.models.formatters import NumeralTickFormatter
from bokeh.models import HoverTool
ic.configureOutput(prefix='ic|',outputFunction=print)
pn.extension('tabulator')

# Vote Coefficients Inputs Dataset

In [22]:
# Read the Vote Coefficients Inputs Dataset
ic("Loading data...")
df_qf = pd.read_csv('./input/vote_coefficients_input.csv', parse_dates=['last_score_timestamp'])
ic(df_qf.shape)

# Drop Unsuccessful Rows
ic(len(df_qf[df_qf['success']==False]))
ic("Dropping unsuccessful data...")
df_qf = df_qf[df_qf['success']==True]
ic(df_qf.shape)

# Drop Unecessary Columns
drop_columns=['success', 'status', 'type']
ic(drop_columns)
ic("Dropping columns...")
df_qf = df_qf.drop(drop_columns, axis=1)
ic(df_qf.shape)

# Shorten Hash Values for Easier Reading
ic("Shortening hashes...")
df_qf[df_qf.select_dtypes('object').columns] = df_qf.select_dtypes('object').apply(lambda x: np.where(x.str.startswith('0x'), x.str.slice(stop=10), x))

df_qf.head(5)

ic|'Loading data...'
ic|df_qf.shape: (257, 16)
ic|len(df_qf[df_qf['success']==False]): 4
ic|'Dropping unsuccessful data...'
ic|df_qf.shape: (253, 16)
ic|drop_columns: ['success', 'status', 'type']
ic|'Dropping columns...'
ic|df_qf.shape: (253, 13)
ic|'Shortening hashes...'


,id,projectId,applicationId,roundId,token,voter,grantAddress,amount,amountUSD,coefficient,last_score_timestamp,rawScore,threshold
0,0x24a5bbf1,0x64a30a4b,19,0x9E669c0A,0x00000000,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1,2023-04-25 13:48:59.888771+00:00,28.57,15
1,0x3dce13bb,0xc401c980,6,0x9E669c0A,0x00000000,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1,2023-04-25 16:55:55.447871+00:00,27.21,15
2,0x4cf20243,0x97589cd1,7,0x9E669c0A,0x00000000,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1,2023-04-25 17:25:19.667155+00:00,28.57,15
3,0x2b032f10,0xec026845,16,0x9E669c0A,0x00000000,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1,2023-04-25 17:07:33.303578+00:00,23.56,15
4,0x0842753b,0xa9bdf738,29,0x9E669c0A,0x00000000,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1,2023-04-25 17:07:33.303578+00:00,23.56,15


In [23]:
df_qf.describe()

,applicationId,amount,amountUSD,coefficient,rawScore,threshold
count,253.000000,2.530000e+02,253.000000,253.0,253.000000,253.0
mean,16.450593,7.204213e+18,23.990991,1.0,32.417589,15.0
std,8.462483,3.996409e+19,83.592891,0.0,11.695007,0.0
min,2.000000,2.000000e+14,0.368890,1.0,15.220000,15.0
25%,9.000000,3.000000e+15,3.000196,1.0,23.560000,15.0
50%,16.000000,1.000000e+16,5.998856,1.0,29.740000,15.0
75%,24.000000,3.000000e+18,12.346883,1.0,40.480000,15.0
max,29.000000,4.210000e+20,936.545861,1.0,55.060000,15.0


### Exploring data with Tabulator

In [24]:
pn.widgets.Tabulator.theme = 'simple'
pn.widgets.Tabulator(df_qf, layout='fit_data_table', page_size=5)

Tabulator(page_size=5, pagination='local', value=             i...)

### Explore the Vote Coefficients Input Dataset Using Hvplot Explorer

In [25]:
hvexplorer = hvplot.explorer(
    df_qf, 
    height=400)
hvexplorer.param.set_param(kind='step', x='index', y_multi=['amountUSD', 'rawScore'], by=[])
hvexplorer.labels.title = 'TEC Quadratic Funding Round #1 Data'
hvexplorer.labels.xlabel = 'Index'
hvexplorer.labels.ylabel = 'USD Amount and Raw Score'
hvexplorer

hvDataFrameExplorer(axes=Axes(explorer=..., height=400, legend='right', logx=False, logy=False, name='Axes04614', responsive=False, shared_axes=True, width=None, xlim=None, ylim=None), by=[], colormapping=Colormapping(clim=None, cmap='kbc_r', cnorm='linear', color=None, colorbar=None, explorer=..., name='Colormapping04674', rescale_discrete_levels=True, symmetric=False), groupby=[], kind='step', labels=Labels(clabel='', explorer=..., fontscale=1, name='Labels04721', rot=0, title='TEC Quadratic Funding Round #1 Data', xlabel='Index', ylabel='USD Amount and Raw Score'), name='hvDataFrameExplorer04581', operations=Operations(aggregator=None, datashade=False, dynspread=False, explorer=..., name='Operations04765', rasterize=False, x_sampling=None, y_sampling=None), style=Style(alpha=1, explorer=..., name='Style04809'), x='index', y='id', y_multi=['amountUSD', 'rawScore'], z=None)

In [26]:
hvexplorer.param.set_param(kind='scatter', x='rawScore', y_multi=['amountUSD'], by=['projectId'])
hvexplorer.labels.xlabel = 'Raw Score'
hvexplorer.labels.ylabel = 'Amount USD'

### Introducing TE Commons Data

In [27]:
def shorten_hashes(df):
    df[df.select_dtypes('object').columns] = df.select_dtypes('object').apply(lambda x: np.where(x.str.startswith('0x'), x.str.slice(stop=10), x))
    return df

In [28]:
# get table of valid tec holders
# extracted from https://dune.com/queries/2457553/4040451
df_tec = pd.read_csv('./input/tec_holders.csv')
df_tec = shorten_hashes(df_tec)

In [29]:
df_tec

,address,balance,tec_tokens_flag
0,0x38dfd788,150071.717791,1
1,0x5b757549,106053.271906,1
2,0x839395e2,73838.661487,1
3,0xdf290293,69337.513233,1
4,0x45602bfb,59165.981018,1
...,...,...,...
404,0x423d60df,10.873917,1
405,0xc70c7f14,10.674364,1
406,0xae7f1137,10.309472,1
407,0x96bdad64,10.223873,1


In [124]:
# Use the Bokeh Hover Tool to show formatted numbers in the hover tooltip for balances
hover = HoverTool(tooltips=[("address", "@address"), ("balance", "@balance{0.00}")])

# Plot a scatter plot of TEC balances on a logy scale.
df_tec.hvplot.scatter(
    y='balance', 
    yformatter=NumeralTickFormatter(format='0,0'), 
    alpha=0.8, 
    logy=True, 
    hover_cols=['address', 'balance'],
    title="TEC Token Holders Distribution Log Scale",
    tools=[hover],
    size=200,
    color="white",
    line_color="skyblue",
    xlabel="index",
)

:Scatter   [index]   (balance,address)

### Introducing TE Academy Data

In [77]:
# get table of te academy token holders
# extracted from https://dune.com/queries/2457581
df_tea_dune = pd.read_csv('./input/tea_holders_dune.csv')
df_tea_tea = pd.read_excel('./input/tea_holders_tea.xlsx')

# Combine
df_tea = pd.concat([df_tea_dune, df_tea_tea]).drop_duplicates(subset=['wallet'])

# Shorten hashes for convenience
df_tea = shorten_hashes(df_tea)

# Make a contiguous index
df_tea = df_tea.reset_index(drop=True)

# Fill balance of TEA with -1 for now
df_tea = df_tea.fillna(-1)

In [78]:
df_tea

,wallet,balance,tea_flag
0,0x68f6f2db,1.0,1
1,0x3e0cf03f,5.0,1
2,0x1d1874f9,1.0,1
3,0x4daa278b,3.0,1
4,0xc710f3da,3.0,1
...,...,...,...
244,0xe1954808,-1.0,1
245,0x7f990adf,-1.0,1
246,0xd1595177,-1.0,1
247,0xcc449df4,-1.0,1


In [126]:
df_tea.hvplot.scatter(y='balance', x='index', title="TEA Credentials Balances Scatter Plot", alpha=0.8)

:Scatter   [index]   (balance)

In [127]:
df_tea.groupby('balance').count().hvplot.bar(y='wallet', title="TEA Credentials Balances Bar Chart", ylabel="Wallet Count", alpha=0.8)

:Bars   [balance]   (wallet)

# Calculate Coefficients

In [136]:
# Drop unecessary columns
df_coef = df_qf.drop(columns=['roundId', 'threshold', 'token', 'last_score_timestamp'])
df_coef

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1,28.57
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1,27.21
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1,28.57
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1,23.56
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1,23.56
...,...,...,...,...,...,...,...,...,...
252,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1,29.74
253,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1,21.07
254,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1,21.07
255,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1,18.04


In [137]:
# Left join the three tables
df_merged = df_coef.merge(
    df_tec, left_on='voter', right_on='address',how='left').merge(
    df_tea, left_on='voter', right_on='wallet',how='left', suffixes=('_tec', '_tea')).drop(columns=['address','wallet'])
df_merged.sample(5)

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag
81,0xd56f22b3,0xe6424ab2,22,0xdfbecc0b,0x8110d1D0,1.000000e+18,0.998774,1,17.83,NaN,NaN,NaN,NaN
56,0xd56698d4,0xe8249a10,14,0x468fd68b,0x9b55D80A,1.000000e+18,1.001181,1,26.79,NaN,NaN,NaN,NaN
80,0x365c3b2e,0xe8249a10,14,0xb2d60143,0x9b55D80A,2.500000e+19,24.969361,1,33.26,5283.368339,1.0,3.0,1.0
231,0x28ccd407,0x4cd41869,25,0x94e9b636,0xBEC643BD,3.240000e+16,59.769901,1,18.77,NaN,NaN,NaN,NaN
32,0x24badc10,0xf1f4942d,24,0xcd0d4cdb,0x4f8c531d,6.000000e+18,6.005987,1,37.04,NaN,NaN,NaN,NaN


In [138]:
# Replace Nan values with 0
df_merged = df_merged.fillna(0)

# Multiply coefficient by 1.5 if tec_flag or tea_flag = 1
df_merged['coefficient'] = 1 + 0.5 * (df_merged['tec_tokens_flag'].astype(int) | df_merged['tea_flag'].astype(int))
df_merged

,id,projectId,applicationId,voter,grantAddress,amount,amountUSD,coefficient,rawScore,balance_tec,tec_tokens_flag,balance_tea,tea_flag
0,0x24a5bbf1,0x64a30a4b,19,0x9ba96198,0xA26d6AEB,5.000000e+15,9.184332,1.5,28.57,0.0,0.0,3.0,1.0
1,0x3dce13bb,0xc401c980,6,0x9390fa86,0x9390fA86,2.200000e+15,4.094567,1.0,27.21,0.0,0.0,0.0,0.0
2,0x4cf20243,0x97589cd1,7,0x5136cdfc,0x0035cC37,4.000000e+16,74.446665,1.0,28.57,0.0,0.0,0.0,0.0
3,0x2b032f10,0xec026845,16,0x524cb61b,0x45b79C6b,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0
4,0x0842753b,0xa9bdf738,29,0x524cb61b,0x5041A1C1,3.000000e+15,5.583500,1.0,23.56,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,0x26e1e300,0x97589cd1,7,0x4405f427,0x0035cC37,1.000000e+15,1.847803,1.0,29.74,0.0,0.0,0.0,0.0
249,0xa21ca1aa,0xec026845,16,0xcdfbbe10,0x45b79C6b,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0
250,0x634b5156,0xf1f4942d,24,0xcdfbbe10,0x4f8c531d,1.000000e+15,1.843793,1.0,21.07,0.0,0.0,0.0,0.0
251,0x4efa29aa,0xcf3165f4,10,0x410d86e3,0x7f3eb18E,1.000000e+15,1.843793,1.0,18.04,0.0,0.0,0.0,0.0


In [140]:
df_merged.to_csv('output/TEGR1.csv', index=False)

# Statistics

In [131]:
df_merged = df_merged.replace(0,np.nan)

In [132]:
# some simple statistics on the left join
df_merged[['id','tec_tokens_flag','tea_flag']].count()

id                 253
tec_tokens_flag    108
tea_flag            44
dtype: int64

In [133]:
# count the number of unique voters
df_merged[['voter','tec_tokens_flag','tea_flag']].drop_duplicates().count()

voter              83
tec_tokens_flag    19
tea_flag            8
dtype: int64

In [134]:
# count the number of voters that have both tec and tea tokens
df_merged[(df_merged['tec_tokens_flag']==True) & (df_merged['tea_flag']==True)][['voter','tec_tokens_flag','tea_flag']].drop_duplicates().count()

voter              4
tec_tokens_flag    4
tea_flag           4
dtype: int64

### Use Langchain for Data Science Help

In [44]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.llms import OpenAI

agent = create_pandas_dataframe_agent(OpenAI(temperature=0), df_qf, verbose=False)

command = agent.run("List columns that only have one value. Return a dataframe that removes those columns. Return the python command to achieve this.")

In [45]:
command

"df.drop(columns=['coefficient', 'roundId', 'threshold'])"